<a href="https://colab.research.google.com/github/MuhammedShiway/TEKNOFEST-2023/blob/main/CROP_ROI_RESIZE_384_%5BComplete%5D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
folder_path = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/PNG1024"
f512 = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/384/ALL"

train_dir = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4"
#val_dir = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4/VAL"
#c_train_dir = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4/CTRAIN"
#c_val_dir = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4/CVAL"

xl_path = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/tabdata.xlsx"

In [ ]:
aug_train = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4/AUG TRAIN"
aug_val = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/1024/JUST 4/AUG VAL"

# 384 RESIZE

In [ ]:
for f in os.listdir(folder_path):
  patient_id = f
  for a in os.listdir(os.path.join(folder_path, f)):
    img_type = a
    curr_path = os.path.join(folder_path, patient_id, img_type)
    new_path = os.path.join(f512, (str(patient_id) + img_type))

    image = Image.open(curr_path)
    
    # Resize the image+
    res_image = image.resize((384, 384))
    # Save the resized image
    res_image.save(new_path)
print("OPERATION COMPLETE")


OPERATION COMPLETE


In [ ]:
os.listdir(aug_val)

['BI-RADS0', 'BI-RADS1-2', 'BI-RADS4-5']

# 384 Resize for AUG

In [ ]:
from PIL import Image

for i in os.listdir(aug_train):
  for j in os.listdir(os.path.join(aug_train, i)):
    # We now have the patient folders
    for img in os.listdir(os.path.join(aug_train, i, str(j))):
      img_path = os.path.join(aug_train, i, str(j), img)
      image = Image.open(img_path)

      # Resize the image
      res_image = image.resize((384, 384))
      # Save the resized image
      res_image.save(img_path)


In [ ]:
# Load dataset
data = pd.read_excel(xl_path)

In [ ]:
import os
from PIL import Image

# Define the paths to the original image folders and the cropped image folders
train_dir = train_dir
#val_dir = val_dir
ctrain_dir = c_train_dir
cval_dir = c_val_dir

# Define the cropping function
def crop_roi(samp_img, top_perc, right_perc, bottom_perc, left_perc, split, birads, subdir):
    # Define the path to the original image
    image_path = os.path.join(train_dir, split, birads, str(samp_img), subdir + '.png')
    # Define the path to the cropped image
    os.makedirs(os.path.join(ctrain_dir if 'TRAIN' in split else cval_dir, birads, str(samp_img)), exist_ok=True)
    cropped_path = os.path.join(ctrain_dir if 'TRAIN' in split else cval_dir, birads, str(samp_img), subdir + '.png')
    # Load the original image
    img = Image.open(image_path)
    # Get the size of the image
    width, height = img.size
    # Calculate the coordinates of the bounding box based on the percentage values
    left = int(left_perc * width)
    top = int(top_perc * height)
    right = int(width - (right_perc * width))
    bottom = int(height - (bottom_perc * height))
    # Crop the image
    cropped_img = img.crop((left, top, right, bottom))
    # Resize the image to 512x512
    cropped_img = cropped_img.resize((512, 512))
    # Save the cropped image
    cropped_img.save(cropped_path)

# Loop over the TRAIN and VAL folders
for folder in os.listdir(train_dir):
    # Skip any files in the directory
    #if not os.path.isdir(os.path.join(train_dir, folder)):
    #    continue
    # Loop over the subfolders
    for subfolder in os.listdir(os.path.join(train_dir, folder)):
        # Skip any files in the directory
        #if not os.path.isdir(os.path.join(train_dir, folder, subfolder)):
        #    continue
        # Create the corresponding subfolders in CTRAIN or CVAL
        if 'TRAIN' in folder:
            os.makedirs(os.path.join(ctrain_dir, subfolder), exist_ok=True)
            split = 'TRAIN'
        else:
            os.makedirs(os.path.join(cval_dir, subfolder), exist_ok=True)
            split = 'VAL'
        # Loop over the patient folders
        for patient in os.listdir(os.path.join(train_dir, folder, subfolder)):
            # Skip any files in the directory
            if not os.path.isdir(os.path.join(train_dir, folder, subfolder, patient)):
                continue
            # Crop and save the images
            
            crop_roi(patient, 0.15, 0.1, 0.1, 0.1, split, subfolder, 'RCC')
            crop_roi(patient, 0.15, 0.1, 0.1, 0.1, split, subfolder, 'LCC')
            crop_roi(patient, 0.2, 0.1, 0.1, 0.1, split, subfolder, 'RMLO')
            crop_roi(patient, 0.2, 0.1, 0.1, 0.1, split, subfolder, 'LMLO')



